In [1]:
from pyspark import SparkContext, SparkConf

# master URL for the cluster. eg. spark://23.195.26.187:7070  or local
conf = SparkConf().setAppName('simpleApp').setMaster('local')
# create a SparkContext object, which tells Spark how to access a cluster
sc = SparkContext(conf=conf)

## Create Resilient Distributed Datasets (RDDs)
There are two ways to create RDDs: <b>parallelizing</b> an existing collection in your driver program, or referencing a dataset in an <b>external storage system</b>, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.
### Parallelized Collections

In [2]:
data = [1, 2, 3, 4, 5]
# the number of partitions as a second parameter
distData = sc.parallelize(data) 
# Once created, the distributed dataset (distData) can be operated on in parallel.
distData.reduce(lambda a, b: a + b)

15

Spark will run one task for each partition of the cluster. Typically you want 2-4 partitions for each CPU in your cluster. 
### External Datasets: TextFiles
External Datasets: local file system, HDFS, Cassandra, HBase, Amazon S3, etc. <br>
spark supports text files, SequenceFiles

In [3]:
# hdfs:///tmp/lxw1234/1.txt
# file:///etc/hadoop/conf/core-site.xml
# second argument for controlling the number of partitions of the file.
distFile = sc.textFile("/opt/apache-spark/README.md")
# add up the sizes of all the lines
distFile.map(lambda s: len(s)).reduce(lambda a, b: a + b)

3706

* <b>If using a path on the local filesystem, the file must also be accessible at the same path on worker nodes. Either copy the file to all workers or use a network-mounted shared file system.</b>

* All of Spark’s file-based input methods, including textFile, support running on directories, compressed files, and wildcards as well. For example, you can use textFile("/my/directory"), textFile("/my/directory/*.txt"), and textFile("/my/directory/*.gz").

* The textFile method also takes an optional second argument for controlling the number of partitions of the file. By default, Spark creates one partition for each block of the file (blocks being 128MB by default in HDFS), but you can also ask for a higher number of partitions by passing a larger value. Note that you cannot have fewer partitions than blocks.

* <b>SparkContext.wholeTextFiles</b> lets you read a directory containing multiple small text files, and returns each of them as (filename, content) pairs. This is in contrast with textFile, which would return one record per line in each file.

* <b>RDD.saveAsPickleFile</b> and <b>SparkContext.pickleFile</b> support saving an RDD in a simple format consisting of pickled Python objects. Batching is used on pickle serialization, with default batch size 10.

* SequenceFile and Hadoop Input/Output Formats

### External Datasets: SequenceFiles

In [4]:
rdd = sc.parallelize(range(1, 4)).map(lambda x: (x, "a" * x))
rdd.saveAsSequenceFile("path/to/file")
sorted(sc.sequenceFile("path/to/file").collect())

[(1, 'a'), (2, 'aa'), (3, 'aaa')]

### External Datasets: Hadoop Input/Output Formats

In [5]:
# conf = {"es.resource": "index/type"}
# rdd = sc.newAPIHadoopRDD("org.elasticsearch.hadoop.mr.EsInputFormat",
#                              "org.apache.hadoop.io.NullWritable",
#                              "org.elasticsearch.hadoop.mr.LinkedMapWritable",
#                              conf=conf)
# rdd.first()  # the result is a MapWritable that is converted to a Python dict

### External Datasets: Cassandra Input/Output Formats

If you have custom serialized binary data (such as loading data from Cassandra / HBase), then you will first need to transform that data on the Scala/Java side to something which can be handled by Pyrolite’s pickler. A <a href="https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.api.python.Converter">Converter</a> trait is provided for this. Simply extend this trait and implement your transformation code in the convert method. Remember to ensure that this class, along with any dependencies required to access your InputFormat, are packaged into your Spark job jar and included on the PySpark classpath.

### External Datasets: HBase Input/Output Formats

See the <a href="https://github.com/apache/spark/tree/master/examples/src/main/python">Python examples</a> and 
<a href="https://github.com/apache/spark/blob/master/examples/src/main/scala/org/apache/spark/examples/pythonconverters/AvroConverters.scala">Converter examples</a> for examples of using Cassandra / HBase InputFormat and OutputFormat with custom converters.

## RDD Operations
* <b>transformations</b>, which create a new dataset from an existing one (all transformations in Spark are <b>lazy</b>). (eg. <b>map</b>)
* <b>actions</b>, which return a value to the driver program after running a computation on the dataset. (eg. <b>reduce</b>)

By default, each transformed RDD may be recomputed each time you run an action on it. However, you may also <b>persist</b> an RDD in memory using the <b>persist</b> (or <b?cache</b>) method, 

In [6]:
lines = sc.textFile('data/data.txt')
# It is lazy, starting the computation when reduce operate started
lineLengths = lines.map(lambda s: len(s))
# It should be added before the reduce
# save in memory after the first time it is computed
lineLengths.persist()    
totalLength = lineLengths.reduce(lambda a, b: a+b)

## Passing Functions to Spark
* Lambda expressions
* Local defs inside the function calling into Spark, for longer code.
* Top-level functions in a module.

In [7]:
"""MyScript.py"""
if __name__ == "__main__":
    def myFunc(s):
        words = s.split(" ")
        return len(words)

    # sc = SparkContext(...)
    sc.textFile("file.txt").map(myFunc)

In [8]:
#  it is also possible to pass a reference to a method in a class instance
class MyClass(object):
    def func(self, s):
        return s
    def doStuff(self, rdd):
        return rdd.map(self.func)

## Understanding closures
One of the harder things about Spark is understanding the scope and life cycle of variables and methods when executing code across a cluster. RDD operations that modify variables outside of their scope can be a frequent source of confusion. 

In [9]:
counter = 0
rdd = sc.parallelize(data)

# Wrong: Don't do this!!
def increment_counter(x):
    global counter
    counter += x
rdd.foreach(increment_counter)

print("Counter value: ", counter)

Counter value:  0


To execute jobs, Spark breaks up the processing of RDD operations into tasks, each of which is executed by an <b>executor</b>. This closure is serialized and sent to each executor.<br>
In general, <b>closures - constructs like loops or locally defined methods, should not be used to mutate some global state</b>. Spark does not define or guarantee the behavior of mutations to objects referenced from outside of closures. Some code that does this may work in <b>local mode</b>, but that’s just by accident and such code will not behave as expected in <b>distributed mode</b>. Use an <b>Accumulator</b> instead if some global aggregation is needed.

## Printing elements of an RDD

in cluster mode, the output to stdout being called by the executors is now writing to the executor’s stdout instead, not the one on the driver, so stdout on the driver won’t show these! To print all elements on the driver, one can use the collect() method to first bring the RDD to the driver node thus: <b>rdd.collect().foreach(println)</b>. This can cause the driver to run <b>out of memory</b>, though, because collect() fetches the entire RDD to a single machine; if you only need to print a few elements of the RDD, a safer approach is to use the take(): <b>rdd.take(100).foreach(println)</b>.

## Working with Key-Value Pairs

In [10]:
lines = sc.textFile('data/data.txt')
pairs = lines.map(lambda s: (s, 1))
counts = pairs.reduceByKey(lambda a, b: a + b)
# counts.sortByKeys()
# counts.collect() to bring them back to the driver program as a list of objects

## Transformations
| Transformation | Meaning |
| :-- | :-- |
| map(func) | |
| filter(func) | on which func returns true. |
| flatMap(func) | Similar to map, but each input item can be mapped to 0 or more output items (so func should return a Seq rather than a single item). |
| mapPartitions(func) | |
| mapPartitionsWithIndex(func) | func with an integer value representing the index of the partition |
| sample(withReplacement, fraction, seed) | Sample a fraction fraction of the data, with or without replacement, using a given random number generator seed. |
| union(otherDataset) | |
| intersection(otherDataset) | |
| distinct([numTasks])) | Return a new dataset that contains the distinct elements of the source dataset. |
| groupByKey([numTasks]) |  When called on a dataset of (K, V) pairs, returns a dataset of (K, Iterable<V>) pairs. Note: If you are grouping in order to perform an aggregation (such as a sum or average) over each key, using <b>reduceByKey</b> or <b>aggregateByKey</b> will yield much better performance. Note: By default, the level of parallelism in the output depends on the number of partitions of the parent RDD. You can pass an optional numTasks argument to set a different number of tasks.|
| reduceByKey(func, [numTasks]) | |
| aggregateByKey(zeroValue)(seqOp, combOp, [numTasks]) | |
| sortByKey([ascending], [numTasks]) | sorted by keys in ascending or descending order, as specified in the <b>boolean</b> ascending argument. |
| join(otherDataset, [numTasks]) | 	When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (V, W)) pairs with all pairs of elements for each key. Outer joins are supported through <b>leftOuterJoin, rightOuterJoin, and fullOuterJoin</b>.|
| cogroup(otherDataset, [numTasks]) | When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (Iterable<V>, Iterable<W>)) tuples. |
| cartesian(otherDataset) | When called on datasets of types T and U, returns a dataset of (T, U) pairs (all pairs of elements). |
| pipe(command, [envVars]) | Pipe each partition of the RDD through a shell command, e.g. a Perl or bash script. RDD elements are written to the process's stdin and lines output to its stdout are returned as an RDD of strings. |
| coalesce(numPartitions) | Decrease the number of partitions in the RDD to numPartitions. Useful for running operations more efficiently after filtering down a large dataset. |
| repartition(numPartitions) | |
| repartitionAndSortWithinPartitions(partitioner) | |

## Actions

| Actions | Meaning |
| :-- | :-- |
| reduce(func) | |
| collect() |	Return all the elements of the dataset as an array at the driver program. This is usually useful after a filter or other operation that returns a sufficiently small subset of the data. |
| count() | |
| first() | |
| take(n) | |
| takeSample(withReplacement, num, [seed]) | |
| takeOrdered(n, [ordering]) | |
| saveAsTextFile(path) | 	Write the elements of the dataset as a text file (or set of text files) in a given directory in the local filesystem, HDFS or any other Hadoop-supported file system. Spark will call toString on each element to convert it to a line of text in the file. |
| countByKey() | |
| foreach(func) | 	Run a function func on each element of the dataset. This is usually done for side effects such as updating an Accumulator or interacting with external storage systems. Note: modifying variables other than Accumulators outside of the foreach() may result in undefined behavior. See Understanding closures for more details. |

## Shuffle operations
The shuffle is Spark’s mechanism for re-distributing data so that it’s grouped differently across partitions. <br>
to organize all the data for a single <b>reduceByKey</b> reduce task to execute, Spark needs to perform an all-to-all operation. It must read from all partitions to find all the values for all keys, and then bring together values across partitions to compute the final result for each key - this is called the <b>shuffle</b>.<br>
If one desires predictably ordered data following shuffle then it’s possible to use:

* mapPartitions to sort each partition using, for example, .sorted
* repartitionAndSortWithinPartitions to efficiently sort partitions while simultaneously repartitioning
* sortBy to make a globally ordered RDD

Operations which can cause a shuffle include <b>repartition</b> operations like repartition and coalesce, ‘<b>ByKey</b> operations (except for counting) like <b>groupByKey</b> and <b>reduceByKey</b>, and <b>join</b> operations like cogroup and join.

## RDD Persistence
The available storage levels in Python include <b>MEMORY_ONLY, MEMORY_ONLY_2, MEMORY_AND_DISK, MEMORY_AND_DISK_2, DISK_ONLY, and DISK_ONLY_2.</b>

## Removing Data
<b>RDD.unpersist() </b>

## Shared Variables
Normally, when a function passed to a Spark operation (such as map or reduce) is executed on a remote cluster node, it works on separate copies of all the variables used in the function. These variables are copied to each machine, and no updates to the variables on the remote machine are propagated back to the driver program. Supporting general, read-write shared variables across tasks would be inefficient. However, Spark does provide two limited types of <b>shared variables</b> for two common usage patterns: <b>broadcast variables</b> and <b>accumulators</b>.



### broadcast variables
